Just do it.

Attachments:
- CrackMe.exe 
- CrackMe.txt

```
$ wget https://quals.2018.volgactf.ru/files/fb8e084723ececc80d62516abc53fb46/CrackMe.exe
$ file CrackMe.exe
CrackMe.exe: PE32 executable (console) Intel 80386 Mono/.Net assembly, for MS Windows

$ wget https://quals.2018.volgactf.ru/files/0f470ab2c54b60a74de77296be6bb841/CrackMe.txt
$ hexdump -C CrackMe.txt
00000000  76 60 fe 32 69 da 17 bd  b3 0e ec 07 a2 41 35 19  |v`.2i........A5.|
00000010  2f 5d 9a ef 15 f7 07 86  85 45 5c 88 bd cb 14 d9  |/].......E\.....|
00000020  49 21 bb 9c 7d 7f b0 65  5a ae a1 02 53 a8 ac 91  |I!..}..eZ...S...|
00000030  e9 eb 82 46 a5 88 25 59  4a 0b 35 48 33 5d ef e2  |...F..%YJ.5H3]..|
00000040
```

Open `CrackMe.exe` in [dotpeek](https://www.jetbrains.com/decompiler/). There, we find:

```c#
internal class Program
{
  private static void Main(string[] args)
  {
    [...]
    CryptoOperation crypto = new CryptoOperation();
    crypto.FileName = path;
    crypto.UserPassword = str1;
    [...]
    Program.Decrypt(crypto, outputFile);
  }

  private static void Decrypt(CryptoOperation crypto, string outputFile)
  {
    crypto.ParseFileToDecrypt();
    byte[] buffer = crypto.DecryptFile();
    [...]
  }
}

internal class CryptoOperation
{
  [...]
  public string UserPassword
  {
    set
    {
      this.KeyLength = 16;
      this.UserKey = this.CombineKeys(MD5.Create().ComputeHash(Encoding.UTF8.GetBytes(value)));
    }
  }
  
  private byte[] CombineKeys(byte[] UserKey)
  {
    [see below]
  }
  
  public byte[] DecryptFile()
  {
    byte[] numArray = (byte[]) null;
    using (AesCryptoServiceProvider cryptoServiceProvider = new AesCryptoServiceProvider())
    {
      cryptoServiceProvider.Key = this.UserKey;
      cryptoServiceProvider.IV = this.IV;
      
      [... decrypt with given Key and IV ...]
    }
  }
  
  public void ParseFileToDecrypt()
  {
    byte[] numArray1 = File.ReadAllBytes(this.FileName);
    byte[] numArray2 = new byte[16];
    Array.Copy((Array) numArray1, 0, (Array) numArray2, 0, 16);
    this.IV = numArray2;
    this.ProcessingBytes = new byte[numArray1.Length - 16];
    Array.Copy((Array) numArray1, 16, (Array) this.ProcessingBytes, 0, this.ProcessingBytes.Length);
  }
}
```


We thus learn that the main operation does the following:
```c#
CryptoOperation crypto = new CryptoOperation();
crypto.FileName = path;
crypto.UserPassword = str1; // this calls CombineKeys to derive the Key
crypto.ParseFileToDecrypt(); // this loads the IV and the data from the file
byte[] buffer = crypto.DecryptFile(); // performs actual decryption
```

So let's take a better look at the key derivation function:
```c#
    private byte[] CombineKeys(byte[] UserKey)
    {
      byte[] bytes1 = Encoding.UTF8.GetBytes(new AppSettings().DefaultKey);
      long int64_1 = BitConverter.ToInt64(bytes1, 0);
      long int64_2 = BitConverter.ToInt64(bytes1, 8);
      long int64_3 = BitConverter.ToInt64(UserKey, 0);
      long int64_4 = BitConverter.ToInt64(UserKey, 8);
      long num1 = int64_1 ^ int64_3;
      long num2 = int64_2 ^ int64_4;
      long num3 = ~int64_1 & int64_3 | ~int64_3 & int64_1;
      long num4 = ~int64_2 & int64_4 | ~int64_4 & int64_2;
      int int32_1 = BitConverter.ToInt32(BitConverter.GetBytes(num1), 0);
      int int32_2 = BitConverter.ToInt32(BitConverter.GetBytes(num1), 4);
      int int32_3 = BitConverter.ToInt32(BitConverter.GetBytes(num2), 0);
      BitConverter.ToInt32(BitConverter.GetBytes(num2), 4);
      int num5 = int32_1 >> 2;
      int num6 = int32_2 >> 2;
      int num7 = num5 << 1;
      int num8 = num6 << 1;
      int num9 = num7 << 1;
      int num10 = int32_3 >> 2;
      num10 = num7 << 1;
      int num11 = num9 >> 2;
      int num12;
      int num13;
      if (~(num7 & num11) == (~num7 | ~num11))
      {
        num12 = num8;
        if (~~num3 != num1 && ~~num4 != num2)
          num8 = num11;
        else
          num11 = num8;
        num13 = ~num11;
      }
      else
      {
        num12 = num7;
        if (~~num3 == num1 && ~~num4 == num2)
          num8 = num11;
        else
          num11 = num8;
        num13 = ~num8;
      }
      byte[] bytes2 = BitConverter.GetBytes(~num13);
      byte[] bytes3 = BitConverter.GetBytes(num8);
      byte[] bytes4 = BitConverter.GetBytes(num12);
      byte[] bytes5 = BitConverter.GetBytes(num11);
      byte[] numArray = new byte[16];
      for (int index = 0; index < 4; ++index)
      {
        numArray[index] = bytes2[index];
        numArray[index + 4] = bytes3[index];
        numArray[index + 8] = bytes4[index];
        numArray[index + 12] = bytes5[index];
      }
      return numArray;
    }
```

This looks a bit overwhelming, but it's not too bad. Let's go through the code step by step.

```c#
byte[] UserKey;
byte[] bytes1 = Encoding.UTF8.GetBytes(new AppSettings().DefaultKey); // fixed value: f1k4bv6gsy?MEW!!
```

The code then splits each of the keys up into 64-bit (8-byte) integers, which we will call `U8a`, `U8b`, `D8a` and `D8b`
```
long int64_1 = BitConverter.ToInt64(bytes1, 0);     // D8a
long int64_2 = BitConverter.ToInt64(bytes1, 8);     // D8b
long int64_3 = BitConverter.ToInt64(UserKey, 0);    // U8a
long int64_4 = BitConverter.ToInt64(UserKey, 8);    // U8b
```

```
long num1 = int64_1 ^ int64_3; // D8a ⊕ U8a
long num2 = int64_2 ^ int64_4; // D8b ⊕ U8b
```

```
long num3 = ~int64_1 & int64_3 | ~int64_3 & int64_1;
long num4 = ~int64_2 & int64_4 | ~int64_4 & int64_2;
```

To solve these, it's easiest to build a quick truth table:

| A | B | ~A & B | ~B & A | result |
|---|---|--------|--------|--------|
| 0 | 0 | 0      | 0      | 0      |
| 0 | 1 | 1      | 0      | 1      |
| 1 | 0 | 0      | 1      | 1      |
| 1 | 1 | 0      | 0      | 0      |

so these are just XORs as well:

```
long num3 = ~int64_1 & int64_3 | ~int64_3 & int64_1;  // D8a ⊕ U8a
long num4 = ~int64_2 & int64_4 | ~int64_4 & int64_2;  // D8b ⊕ U8b
```

We now re-interpret all the bytes as 4-byte integers instead. We will call these D4a..D4d and U4a..U4d. Note that D4a and D4b are the first and last 32 bits from D8a.

```
int int32_1 = BitConverter.ToInt32(BitConverter.GetBytes(num1), 0); // D4a ⊕ U4a
int int32_2 = BitConverter.ToInt32(BitConverter.GetBytes(num1), 4); // D4b ⊕ U4b
int int32_3 = BitConverter.ToInt32(BitConverter.GetBytes(num2), 0); // D4c ⊕ U4c
```

Now for some bitfiddling:
```
int num5 = int32_1 >> 2;  // (D4a ⊕ U4a) >> 2
int num6 = int32_2 >> 2;  // (D4b ⊕ U4b) >> 2
int num7 = num5 << 1;     // (D4a ⊕ U4a) >> 2 << 1
int num8 = num6 << 1;     // (D4b ⊕ U4b) >> 2 << 1
int num9 = num7 << 1;     // (D4a ⊕ U4a) >> 2 << 2
int num10 = int32_3 >> 2; // (D4c ⊕ U4c) >> 2
num10 = num7 << 1;        // (D4a ⊕ U4a) >> 2 << 2
int num11 = num9 >> 2;    // (D4a ⊕ U4a) >> 2
```

We end up at a branch:
```c#
if (~(num7 & num11) == (~num7 | ~num11)) {...}
```

Again, using a truth table:

| num7 | num11 | ~(num7 & num11) | (~num7 &#124; ~num11) | equal? |
|------|-------|-----------------|-----------------------|--------|
| 0    | 0     | 1               | 1                     | 1      |
| 0    | 1     | 1               | 1                     | 1      |
| 1    | 0     | 1               | 1                     | 1      |
| 1    | 1     | 0               | 0                     | 1      |

So we always enter this branch.

```c#
num12 = num8;   // (D4b ⊕ U4b) >> 2 << 1
```

Again a branch:
```c#
if (~~num3 != num1 && ~~num4 != num2)   // ( (D8a ⊕ U8a) != (D8a ⊕ U8a) && (D8b ⊕ U8b) != (D8b ⊕ U8b) )
```
This is clearly always false, so we execute

```c#
num11 = num8;   // (D4b ⊕ U4b) >> 2 << 1
```

then
```c#
num13 = ~num11; // ~((D4b ⊕ U4b) >> 2 << 1)
```

The calculated values are then stacked in a byte array:
```c#
byte[] bytes2 = BitConverter.GetBytes(~num13);  // (D4b ⊕ U4b) >> 2 << 1
byte[] bytes3 = BitConverter.GetBytes(num8);    // (D4b ⊕ U4b) >> 2 << 1
byte[] bytes4 = BitConverter.GetBytes(num12);   // (D4b ⊕ U4b) >> 2 << 1 
byte[] bytes5 = BitConverter.GetBytes(num11);   // (D4b ⊕ U4b) >> 2 << 1

byte[] numArray = new byte[16];
for (int index = 0; index < 4; ++index)
{
  numArray[index] = bytes2[index];
  numArray[index + 4] = bytes3[index];
  numArray[index + 8] = bytes4[index];
  numArray[index + 12] = bytes5[index];
}
return numArray;
```      

__Note: The following is how we found the solution, but the argument is not correct. Scroll further down for details__

So that at the end of this, our 16 byte key consists of the same 4-byte value repeated four times. In addition, due to the bitshifting, we know that the highest and lowest bit of this value are 0. This reduces the key space to $2^{30}$, which is large but not insurmountable.

To crack the key, first read the IV and data from the data file:

In [8]:
import requests

crackme_txt = requests.get(
    "https://quals.2018.volgactf.ru/files/0f470ab2c54b60a74de77296be6bb841/CrackMe.txt",
    stream=True).raw.read()

IV = crackme_txt[:16]
data = crackme_txt[16:]

len(data)

48

Then we simply enumerate the keyspace. We know the highest and lowest bit are 0, which means that we want to:

In [9]:
keys = range(
    0b00000000000000000000000000000000,
    0b10000000000000000000000000000000, # not inclusive
                                  0b10
)
len(keys)

1073741824

We then assume the decrypted data starts with "Volga", and try every key:

In [10]:
from Crypto.Cipher import AES

for i in keys:
    key = i.to_bytes(4, "big") * 4
    obj = AES.new(key, AES.MODE_CBC, IV)
    dec = obj.decrypt(data)
    if (dec.startswith(b"Volga")):
        print(key, dec)
        break

b'BDR\xf6BDR\xf6BDR\xf6BDR\xf6' b'VolgaCTF{my_little_cat_solved_this_much_faster}\x01'


__Note: The derivation above is not correct__

MatthiasHeinz [noted](https://github.com/DancingSimpletons/writeups/issues/1) that the above argument about the MSB and LSB being zero is incorrect -- a right shift on a negative number would add an extra 1 bit on the left. While investigating this in more detail, we noticed there was a bigger flaw with the above code: we did not correctly get the keybytes as little endian.

As the right shift will add a bit equal to the old left most bit, the bitmask of the solution should be

`aa?????? ???????? ???????? ???????0`.

However, as [System.BitConverter](https://docs.microsoft.com/en-gb/dotnet/api/system.bitconverter.getbytes?view=netframework-4.7.1#System_BitConverter_GetBytes_System_Int32_) uses the endianness of the system, the key bytes will actually look like

`???????0 ???????? ???????? aa??????`.

For this challenge, we were lucky the solution actually matched both this mask and the flawed

`0??????? ???????? ???????? ???????0`

mask, as the solution was

`01000010 01000100 01010010 11110110`.

To correctly loop over the solutions, we should have generated the following range:

```
C0 00 00 00 ... FF FF FF FE (first two bits 1, last bit 0: -1073741824 ... -2, steps of 2)
00 00 00 00 ... 4F FF FF FE (first two bits 0, last bit 0: 0 ... 1342177278, steps of 2)
```

and this number should have been interpreted as little endian to form the key:

In [3]:
keys = range(-1073741824, 1342177278+1, 2)

int.from_bytes(b'BDR\xf6', 'little', signed=True) in keys

True

In [9]:
for i in keys:
    key = i.to_bytes(4, "little", signed=True) * 4  
    obj = AES.new(key, AES.MODE_CBC, IV)
    dec = obj.decrypt(data)
    if (dec.startswith(b"Volga")):
        print(key, dec)
        break

b'BDR\xf6BDR\xf6BDR\xf6BDR\xf6' b'VolgaCTF{my_little_cat_solved_this_much_faster}\x01'
